# MGMTMSA 403: Optimization

# Assignment 3: Predicting Airbnb Prices

In [1]:
from gurobipy import * 
import numpy as np 
import pandas as pd 
import csv

train_set = "/Users/jeremycao/Downloads/AirbnbTrain.csv"
df = pd.read_csv(train_set)
df.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.103701,-118.332241,1,13,2.0,3,2,150,2,1,100,1,350
1,34.099484,-118.331645,1,8,2.0,2,4,150,1,11,96,1,190
2,34.104321,-118.329662,1,4,1.0,0,1,55,1,1,80,0,85
3,34.101028,-118.317848,0,2,1.0,1,1,20,1,8,98,0,75
4,34.098292,-118.324980,1,2,1.0,1,1,20,1,11,96,0,130


In [2]:
x = np.asarray(df)
x[0,12]

350.0

### MODEL 1

In [3]:
#Model 1
from gurobipy import * 

mod = Model()
m = mod.addVars(12)
z = mod.addVars(1700)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-14


In [4]:
for i in range(1700):
    mod.addConstr(z[i] >= x[i,12] - sum(m[j]*x[i,j] for j in range(12))) 
    mod.addConstr(z[i] >= sum(m[j]*x[i,j] for j in range(12)) - x[i,12])

In [5]:
mod.setObjective(sum(z[i] for i in range(1700)) / 1700, GRB.MINIMIZE) 

mod.update()
mod.optimize()
opt_val1 = mod.objval

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3400 rows, 1712 columns and 41372 nonzeros
Model fingerprint: 0xa0d487df
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve time: 0.02s
Presolved: 3400 rows, 1712 columns, 41372 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 2.995e+04
 Factor NZ  : 3.260e+04 (roughly 2 MB of memory)
 Factor Ops : 4.141e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.13176381e+03  0.00000000e+00  1.36e+03 0.00e+00  2.72e+01     0s
   1   1.42686270e+03  1.34119445e+00  0.00e+00 

In [6]:
beta1 = []
for j in range(12):
    beta1.append(m[j].x)
print(beta1)

[290.25146635184893, 84.03092762130682, 36.783237555536715, 9.936817142125026, 31.694331188789405, 19.70395711447884, 0.0, 0.3106162814102326, 0.0, 0.0, 0.2681158824601206, 5.167454373349855]


In [7]:
test_set = "/Users/jeremycao/Downloads/AirbnbTest.csv"
df2 = pd.read_csv(test_set)
df2.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.100604,-118.341787,0,2,1.0,1,1,40,1,261,96,1,100
1,34.100607,-118.350583,1,8,2.0,2,2,100,2,10,98,0,300
2,34.100610,-118.347617,1,2,1.0,1,1,80,2,1,100,1,125
3,34.100611,-118.342180,1,3,1.0,0,2,55,2,54,97,1,169
4,34.100618,-118.342791,1,4,1.0,1,1,70,2,233,92,1,119


In [8]:
x_test = np.asarray(df2)
error1 = (sum(abs(x_test[i,12] - sum(beta1[j]*x_test[i,j] for j in range(12))) for i in range(699)))/699
print("The predict error is", error1)

The predict error is 35.60453503037783


### MODEL 2

In [9]:
#Model 2
x = np.asarray(df)
x[0,12]

from gurobipy import *

mod = Model()

m = mod.addVars(12)
z = mod.addVars(1700)
p = mod.addVars(12, vtype = GRB.BINARY)
max_bar = 100000

mod.addConstr(sum(p[j] for j in range(12)) <= 3)

for j in range(12):
    mod.addConstr(m[j] <= max_bar*p[j])
    
for i in range(1700):
    mod.addConstr(z[i] >= x[i,12] - sum(m[j]*x[i,j] for j in range(12))) 
    mod.addConstr(z[i] >= sum(m[j]*x[i,j] for j in range(12)) - x[i,12])

mod.setObjective(sum(z[i] for i in range(1700)) / 1700, GRB.MINIMIZE) 

mod.update()
mod.optimize()

opt_val2 = mod.objval

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3413 rows, 1724 columns and 41408 nonzeros
Model fingerprint: 0xe9be68de
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+05]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 828 rows and 414 columns
Presolve time: 0.05s
Presolved: 2585 rows, 1310 columns, 31274 nonzeros
Variable types: 1298 continuous, 12 integer (12 binary)

Root relaxation: objective 3.642625e+01, 1408 iterations, 0.19 seconds (0.36 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.42625    0    5  144.96824   36.42625  74.9%     -    0s
H    0     0      

In [10]:
for j in range(12):
    print(list(df)[j], p[j].x)

latitude 0.0
longitude -0.0
Entire home 1.0
accommodates 1.0
bathrooms -0.0
bedrooms 1.0
beds -0.0
cleaning_fee -0.0
minimum_nights -0.0
number_of_reviews -0.0
review_scores_rating -0.0
instant_bookable 0.0


***The three variables are Entire home, accommodates, and bedrooms***

In [11]:
beta2 = []
for j in range(12):
    beta2.append(m[j].x)
print(beta2)

[0.0, 0.0, 52.00000000000002, 13.99999999999999, 0.0, 32.00000000000001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


***coefficients related to three variables are 52.0, 13.9999 and 32.0***

In [12]:
x_test2 = np.asarray(df2)
error_2 = (sum(abs(x_test2[i,12] - sum(beta2[j]*x_test2[i,j] for j in range(12))) for i in range(699)))/699
print("The predict error of Model 2 is", error_2)

The predict error of Model 2 is 37.73676680972818


### MODEL 3

In [13]:
#Model 3
x = np.asarray(df)
x[0,12]

from gurobipy import * 
mod = Model()

m = mod.addVars(12)
z = mod.addVars(1700)
p = mod.addVars(12, vtype = GRB.BINARY)
bbar = 100000

mod.addConstr(p[6] == 1)
mod.addConstr(sum(p[j] for j in range(12)) <= 3)

for j in range(12): 
    mod.addConstr(m[j] <= bbar*p[j])
    
for i in range(1700):
    mod.addConstr(z[i] >= x[i,12] - sum(m[j]*x[i,j] for j in range(12))) 
    mod.addConstr(z[i] >= sum(m[j]*x[i,j] for j in range(12)) - x[i,12])

mod.setObjective(sum(z[i] for i in range(1700)) / 1700, GRB.MINIMIZE) 

mod.update()
mod.optimize()

opt_val3 = mod.objval

beta3 = []
for j in range(12):
    beta3.append(m[j].x)
print(beta3)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3414 rows, 1724 columns and 41409 nonzeros
Model fingerprint: 0x139764d5
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+05]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 830 rows and 415 columns
Presolve time: 0.04s
Presolved: 2584 rows, 1309 columns, 31271 nonzeros
Variable types: 1298 continuous, 11 integer (11 binary)

Root relaxation: objective 3.642625e+01, 1408 iterations, 0.19 seconds (0.36 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.42625    0    6  144.96824   36.42625  74.9%     -    0s
H    0     0      

In [14]:
for j in range(12):
    print(list(df)[j], p[j].x)

latitude -0.0
longitude 0.0
Entire home 1.0
accommodates 0.0
bathrooms 0.0
bedrooms 1.0
beds 1.0
cleaning_fee 0.0
minimum_nights -0.0
number_of_reviews -0.0
review_scores_rating 0.0
instant_bookable 0.0


***The two other variables are Entire home and bedrooms***

***coefficients related to entire home is 67.875 and bedrooms is 12.125***

***The coeficient that is no longer in Model 3 is accommodates. In my opinion, the entire home and bedrooms are the 2 main features to determine the prices. Moreover, the number of beds (beds) and accomodates are kind of similar features to decide how many people can book a Airbnb. But number of beds are always more important to make the decision. Thus, accomodates was the feature to be dropped.***

In [15]:
x_test3 = np.asarray(df2)
error_3 = (sum(abs(x_test3[i,12] - sum(beta3[j]*x_test3[i,j] for j in range(12))) for i in range(699)))/699
print("The predict error of Model 3 is", error_3)

The predict error of Model 3 is 38.59960658082976
